In [13]:
import pandas as pd


In [14]:
data_df = pd.read_csv('./resources/past_data.csv', index_col=[0], parse_dates=True,infer_datetime_format=True)
data_df.head()

,Rms,Occ,ADR,V1_Food_Amt,V2_Food_Amt,V3_Food_Amt,V4_Food_Amt,V5_Food_Amount,V6_Food_Amount,V7_Food_Amt,V8_Food_Amt,BC_Food_Amount
Period,,,,,,,,,,,,
2022-01-01,"2,346",22.6,206.61,"32,892","43,418","15,651",0,0,"1,933","1,132",0,"158,749"
2022-02-01,"2,882",30.7,227.71,"59,334","54,142","30,849",0,0,"1,832","24,039","4,197","93,428"
2022-03-01,"4,755",45.8,258.37,"80,379","162,710","50,660","11,012","10,053","6,538","12,097","15,639","276,694"
2022-04-01,"4,216",42.0,288.55,"127,933","69,808","56,504","16,564","12,953","7,410","14,228","14,676","185,795"
2022-05-01,"5,359",51.6,295.95,"127,454","143,944","69,528","24,940","20,028","5,573","9,191","25,501","257,015"


In [15]:
# Drop any ',' in any of the columns
data_df = data_df.replace(',','', regex=True)
data_df.head()

,Rms,Occ,ADR,V1_Food_Amt,V2_Food_Amt,V3_Food_Amt,V4_Food_Amt,V5_Food_Amount,V6_Food_Amount,V7_Food_Amt,V8_Food_Amt,BC_Food_Amount
Period,,,,,,,,,,,,
2022-01-01,2346,22.6,206.61,32892,43418,15651,0,0,1933,1132,0,158749
2022-02-01,2882,30.7,227.71,59334,54142,30849,0,0,1832,24039,4197,93428
2022-03-01,4755,45.8,258.37,80379,162710,50660,11012,10053,6538,12097,15639,276694
2022-04-01,4216,42.0,288.55,127933,69808,56504,16564,12953,7410,14228,14676,185795
2022-05-01,5359,51.6,295.95,127454,143944,69528,24940,20028,5573,9191,25501,257015


In [16]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Load data and separate "Rms", "Occ", and "ADR" from "Amt" columns

X = data_df[['Rms', 'Occ', 'ADR', 'BC_Food_Amount']]
y = data_df.drop(['Rms', 'Occ', 'ADR', 'BC_Food_Amount'], axis=1)

# Apply PCA to "Amt" columns and find the number of components that explain 95% of the variance
pca = PCA(n_components=0.95)

# Write the number of components to a variable


pca.fit(y)
y_pca = pca.transform(y)

y_pca


array([[-110336.05810568,   -4257.76378591],
       [ -84974.53096684,   12530.90483239],
       [  23940.53805428,  -15115.0499113 ],
       [ -32685.29739301,   71571.963339  ],
       [  36421.09354545,   40703.3412296 ],
       [ 130176.51034966,  -13831.61209628],
       [ 233454.03055148,   12925.16282669],
       [  70611.13576733,    4096.77428942],
       [   9608.16099885,  -11691.25888658],
       [  68436.59955207,  -51848.6785364 ],
       [ -67139.36605372,   -9109.26861633],
       [ -78727.48561052,   -9203.85854769],
       [ -96205.84247317,  -17887.85376144],
       [-102579.48821618,   -8882.80237519]])

In [17]:
# Combine PCA components with "Rms", "Occ", and "ADR" columns
Xy_pca = pd.concat([X, pd.DataFrame(y_pca, columns=['pca_component_1', 'pca_component_2'], index = X.index)], axis=1)

# Train a linear regression model to predict the PCA components
model = LinearRegression()
model.fit(Xy_pca.drop(['pca_component_1', 'pca_component_2'], axis=1), Xy_pca[['pca_component_1', 'pca_component_2']])
march_data = pd.DataFrame({'Rms': [5074], 'Occ': [48.9], 'ADR': [269.49], "BC_Food_Amount": [215000]})

# Use the model to predict the PCA components for March of 2023
march_pca = model.predict(march_data)
march_pca = pd.DataFrame(march_pca, columns=['pca_component_1', 'pca_component_2'])

# Reverse the PCA transformation to obtain the predicted "Amt" values for March of 2023
march_prediction = pca.inverse_transform(march_pca)
print('Predicted values for March of 2023:', march_prediction)


Predicted values for March of 2023: [[ 96251.28872539 155439.81736217  67831.20365049  20946.99500372
   16417.88023524   5070.18755646  21794.7145908   18817.89214895]]


In [18]:
# Add a row to the dataframe with the predicted values for March of 2023, with the index value of 2023-03-01
march_df = pd.DataFrame(march_prediction, index=[pd.to_datetime('2023-03-01')])


In [19]:
march_df['Rms'] = march_data['Rms'].values
march_df['Occ'] = march_data['Occ'].values
march_df['ADR'] = march_data['ADR'].values
march_df['BC_Food_Amount'] = march_data['BC_Food_Amount'].values


In [20]:
march_df

,0,1,2,3,4,5,6,7,Rms,Occ,ADR,BC_Food_Amount
2023-03-01,96251.288725,155439.817362,67831.20365,20946.995004,16417.880235,5070.187556,21794.714591,18817.892149,5074,48.9,269.49,215000


In [21]:
# Move the columns to the correct order
march_df = march_df[['Rms', 'Occ', 'ADR', 0, 1, 2, 3, 4, 5, 6, 7, 'BC_Food_Amount']]
march_df

,Rms,Occ,ADR,0,1,2,3,4,5,6,7,BC_Food_Amount
2023-03-01,5074,48.9,269.49,96251.288725,155439.817362,67831.20365,20946.995004,16417.880235,5070.187556,21794.714591,18817.892149,215000


In [22]:
march_df.columns = data_df.columns
march_df

,Rms,Occ,ADR,V1_Food_Amt,V2_Food_Amt,V3_Food_Amt,V4_Food_Amt,V5_Food_Amount,V6_Food_Amount,V7_Food_Amt,V8_Food_Amt,BC_Food_Amount
2023-03-01,5074,48.9,269.49,96251.288725,155439.817362,67831.20365,20946.995004,16417.880235,5070.187556,21794.714591,18817.892149,215000


In [23]:
# Concatenate the original dataframe with the predicted dataframe
final_df = pd.concat([data_df, march_df])
final_df

,Rms,Occ,ADR,V1_Food_Amt,V2_Food_Amt,V3_Food_Amt,V4_Food_Amt,V5_Food_Amount,V6_Food_Amount,V7_Food_Amt,V8_Food_Amt,BC_Food_Amount
2022-01-01,2346,22.6,206.61,32892,43418,15651,0,0,1933,1132,0,158749
2022-02-01,2882,30.7,227.71,59334,54142,30849,0,0,1832,24039,4197,93428
2022-03-01,4755,45.8,258.37,80379,162710,50660,11012,10053,6538,12097,15639,276694
2022-04-01,4216,42.0,288.55,127933,69808,56504,16564,12953,7410,14228,14676,185795
2022-05-01,5359,51.6,295.95,127454,143944,69528,24940,20028,5573,9191,25501,257015
2022-06-01,7396,73.6,291.47,118708,248406,86643,41157,27636,5569,30492,21695,446979
2022-07-01,8631,83.1,346.15,170409,319692,145546,56226,44676,5762,13896,39720,171302
2022-08-01,6254,60.2,285.30,108896,184765,80091,30740,25556,5502,54852,24521,332870
2022-09-01,4709,46.9,282.57,71079,142417,57210,17273,13628,5921,39789,28617,232813
2022-10-01,5476,52.7,247.58,72582,218369,53934,6288,6192,7372,31118,16519,447234


In [61]:
final_df.to_excel('./resources/predicted_data.xlsx')

In [62]:
final_df = final_df.astype(float)


In [64]:
# Create a total food column that is the sum of all of the columns except for "Rms", "Occ", and "ADR"
final_df['Total Food'] = final_df.drop(['Rms', 'Occ', 'ADR'], axis=1).sum(axis=1)


In [65]:
final_df

,Rms,Occ,ADR,V1_Food_Amt,V2_Food_Amt,V3_Food_Amt,V4_Food_Amt,V5_Food_Amount,V6_Food_Amount,V7_Food_Amt,V8_Food_Amt,BC_Food_Amount,Total Food
2022-01-01,2346.0,22.6,206.61,32892.000000,43418.00000,15651.000000,0.00000,0.000000,1933.000000,1132.000000,0.000000,158749.000000,2.537750e+05
2022-02-01,2882.0,30.7,227.71,59334.000000,54142.00000,30849.000000,0.00000,0.000000,1832.000000,24039.000000,4197.000000,93428.000000,2.678210e+05
2022-03-01,4755.0,45.8,258.37,80379.000000,162710.00000,50660.000000,11012.00000,10053.000000,6538.000000,12097.000000,15639.000000,276694.000000,6.257820e+05
2022-04-01,4216.0,42.0,288.55,127933.000000,69808.00000,56504.000000,16564.00000,12953.000000,7410.000000,14228.000000,14676.000000,185795.000000,5.058710e+05
2022-05-01,5359.0,51.6,295.95,127454.000000,143944.00000,69528.000000,24940.00000,20028.000000,5573.000000,9191.000000,25501.000000,257015.000000,6.831740e+05
2022-06-01,7396.0,73.6,291.47,118708.000000,248406.00000,86643.000000,41157.00000,27636.000000,5569.000000,30492.000000,21695.000000,446979.000000,1.027285e+06
2022-07-01,8631.0,83.1,346.15,170409.000000,319692.00000,145546.000000,56226.00000,44676.000000,5762.000000,13896.000000,39720.000000,171302.000000,9.672290e+05
2022-08-01,6254.0,60.2,285.30,108896.000000,184765.00000,80091.000000,30740.00000,25556.000000,5502.000000,54852.000000,24521.000000,332870.000000,8.477930e+05
2022-09-01,4709.0,46.9,282.57,71079.000000,142417.00000,57210.000000,17273.00000,13628.000000,5921.000000,39789.000000,28617.000000,232813.000000,6.087470e+05
2022-10-01,5476.0,52.7,247.58,72582.000000,218369.00000,53934.000000,6288.00000,6192.000000,7372.000000,31118.000000,16519.000000,447234.000000,8.596080e+05
